In [ ]:
# Connect to drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
!pip install ghostscript
!pip install camelot-py[cv]
!pip install excalibur-py
!apt install ghostscript python3-tk
!pip install tika
!pip install pypdf
!pip install python-dotenv
!pip install 'PyPDF2<3.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 986.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import requests
import camelot
import json
import logging
import re
import pandas as pd
import os
from tika import parser
from pypdf import PdfReader
import dotenv
dotenv.load_dotenv('.env')


True

In [4]:
DATA_FOLDER = os.environ.get('DATA_FOLDER')

In [5]:
# A check to view downloaded files
lei_files = os.listdir(DATA_FOLDER+'/Leading Economic Indicators')
lei_files = [i for i in lei_files if '.pdf' in i]


In [6]:
def get_table_page(lei_path):
  reader = PdfReader(lei_path)
  text = ""

  search_text1 = "TABLE 1:[\w\s]*KENYA CONSUMER PRICE INDICES"


  search_text3 = "TABLE 1(B): KENYA CONSUMER PRICE INDICES"
  count = 0
  x = []
  counts = []

  for page in reader.pages:
      page_extract = page.extract_text().upper().replace('  ',' ').replace('  ',' ')
      count+=1
      if len(x)==0:
        try:
            x1 = re.search(search_text1, page_extract)
            if x1 and count>3:
                return count
        except:
            x1 = re.search(search_text2, page_extract)
            if x1 and count>3:
                counts.append(count)

      else:
        x2 = re.search(search_text3, page_extract)
        if x2 and count>3:
            counts.append(count)
            return counts






In [7]:
def get_headers(lei_path, table_page):
  page = str(table_page)
  header_catch = 'OVERALL MONTH ON MONTH INFLATION'
  header_row = 0
  lattice_tables_list = camelot.read_pdf(lei_path, pages=page, flavor='lattice', strip_text='\n')

  if len(lattice_tables_list)>0:
    lattice_table = lattice_tables_list[0].df
    for idx, row in lattice_table.iterrows():
      headers_caught = [i for i in row if header_catch in i.upper()]
      if len(headers_caught)>0:
        header_row=idx

    headers_table = camelot.read_pdf(lei_path, pages=page, flavor='lattice', strip_text='\n')[0].df
    headers = list(headers_table.iloc[header_row])
    return headers
  else:
    return 'No lattice table'


In [8]:
def get_table_page_tmp(lei_path):
  reader = PdfReader(lei_path)
  search_text1 = "TABLE1:[\w]*CONSUMERPRICEINDICES"
  search_text2 = "TABLE1\(A\):[KENYA]*CONSUMERPRICEINDICES"
  search_text3 = "TABLE1\(B\):[KENYA]*INFLATIONRATE"

  count = 0
  table_page_dict = {}
  counts = []

  for page in reader.pages:
      page_extract = page.extract_text().upper().replace(' ','')
      count+=1
      if len(table_page_dict)==0:
        cpi_inf_match = re.search(search_text1, page_extract)
        if cpi_inf_match and count>3:
          table_page_dict.update({'cpi_inf_match':count})
        else:
          cpi_match = re.search(search_text2, page_extract)
          if cpi_match and count>3:
            table_page_dict.update({'cpi_match':count})

      if 'cpi_match' in table_page_dict:
        inf_match = re.search(search_text3, page_extract)
        if inf_match and count>3:
            table_page_dict.update({'inf_match':count})
  return table_page_dict

In [16]:
cpi_inf_page_list = []
for file_no,lei_file in enumerate(lei_files):
  lei_path = DATA_FOLDER+'/Leading Economic Indicators/'+lei_file
  table_pages = get_table_page_tmp(lei_path)
  cpi_inf_page_list.append(table_pages)

In [62]:
def get_cpi_inf_data(lei_path, table_page):
  df = camelot.read_pdf(lei_path, pages=table_page, flavor='stream', strip_text=' \n')[0].df
  header_catch = 'OVERALL MONTH ON MONTH INFLATION'
  header_row = 1
  lattice_table = camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df
  for idx, row in lattice_table.iterrows():
    if header_catch in row:
      header_row=idx

  headers = list(camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df.iloc[header_row])
  df.columns = [i.replace('*','').replace('  ',' ') for i in headers]
  df = df[df[headers[0]]!='']
  # Take out years

  years = [i for i in list(df['PERIOD'].values) if '2'==i[0]]
  df = df[df['PERIOD'].str.slice(0,1)!='2']
  period_list = list(df['PERIOD'].values)
  year_count=0
  period_years=[]

  for k,i in enumerate(period_list):
    period_years.append(i+' '+years[year_count])
    if (k+1)<len(period_list):
      if period_list[k+1]=='January':
        year_count+=1
  df['PERIOD']=period_years
  df = df.dropna(how='any')
  df = df[pd.to_datetime(df['PERIOD'], errors='coerce').notnull()]
  return df

In [63]:
lei_path = DATA_FOLDER+'/Leading Economic Indicators/'+lei_files[178]

if "cpi_inf_match" in cpi_inf_page_list[178]:
  table_page = str(cpi_inf_page_list[178]['cpi_inf_match'])
  df = get_cpi_inf_data(lei_path, table_page)
df

<ipython-input-62-766a373d16d7>:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df[pd.to_datetime(df['PERIOD'], errors='coerce').notnull()]


,PERIOD,NAIROBI INDEX,REST OF URBAN TOWNS INDEX,OVERALL KENYA INDEX,OVERALL MONTH ON MONTH INFLATION,UNDERLYING INFLATION RATE
6,June 2007,224.91,224.95,224.93,11.1,5.2
7,July 2007,226.53,226.60,226.57,13.6,5.6
8,August 2007,227.64,224.56,225.79,12.4,5.2
9,September 2007,230.29,226.74,228.16,11.7,5.1
10,October 2007,230.93,229.37,229.99,10.6,5.2
11,November 2007,233.73,232.99,233.28,11.8,5.8
12,December 2007,242.14,238.26,239.81,12.0,5.9
14,January 2008,266.28,257.39,260.94,18.2,7.6
15,February 2008,267.04,265.92,266.37,19.1,9.3
16,March 2008,277.61,274.40,274.55,21.8,9.8
